# Store Sales

## Packages/Set up

In [ ]:
import numpy as np
import pandas as pd
import plotly as pltly
import plotly.express as px
import matplotlib.pyplot as plt
import lightgbm as lgb
import statsmodels.api as sm

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

import glob
import os
import itertools

## Load Data

In [ ]:
# List files in data directory
mypath = '
file_list = []
for file in glob.glob(f'{mypath}/*.csv'):
    file_list.append(file)
file_list

In [ ]:
holiday_events = pd.read_csv(
    '/home/samc/BDO-Projects/store-sales-time-series-forecasting/holidays_events.csv',
    dtype={
        'type': 'category',
        'locale': 'category',
        'locale_name': 'category',
        'description': 'object',
        'transfered': 'bool'
    },
    parse_dates=['date'],
    infer_datetime_format=True
)

# transactions = pd.read_csv(
#     '/home/samc/BDO-Projects/store-sales-time-series-forecasting/transactions.csv',
#     dtype={
#         'store_nbr': 'category',
#         'transactions': 'int64',
#     },
#     parse_dates=['date'],
#     infer_datetime_format=True
# )

train = pd.read_csv(
    '/home/samc/BDO-Projects/store-sales-time-series-forecasting/train.csv',
    dtype={
        'id': 'int64',
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float64',
        'onpromotion': 'int64'
    },
    parse_dates=['date'],
    infer_datetime_format=True
)

oil = pd.read_csv(
    '/home/samc/BDO-Projects/store-sales-time-series-forecasting/oil.csv',
    dtype={
        'dcoilwtico': 'float64'
    },
    parse_dates=['date'],
    infer_datetime_format=True   
)

test = pd.read_csv(
    '/home/samc/BDO-Projects/store-sales-time-series-forecasting/test.csv',
    dtype={
        'id': 'int64',
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float64',
        'onpromotion': 'int64'
    },
    parse_dates=['date'],
    infer_datetime_format=True
)

stores = pd.read_csv(
    '/home/samc/BDO-Projects/store-sales-time-series-forecasting/stores.csv',
    dtype={
        'store_nbr': 'category',
        'city': 'category',
        'state': 'category',
        'type': 'category',
        'cluster': 'category'
    },
    header=0,
    names = [
        'store_nbr',
        'city',
        'state',
        'store_type',
        'cluster'
    ]
)

In [ ]:
# Create date variables
train['year'] = pd.DatetimeIndex(train['date']).year.astype('category')
train['month'] = pd.DatetimeIndex(train['date']).month.astype('category')
train['day'] = pd.DatetimeIndex(train['date']).day.astype('category')
train['dayofweek'] = pd.DatetimeIndex(train['date']).dayofweek.astype('category')

test['year'] = pd.DatetimeIndex(test['date']).year.astype('category')
test['month'] = pd.DatetimeIndex(test['date']).month.astype('category')
test['day'] = pd.DatetimeIndex(test['date']).day.astype('category')
test['dayofweek'] = pd.DatetimeIndex(test['date']).dayofweek.astype('category')

## Data Summary

In [ ]:
holiday_events.info()

In [ ]:
# transactions.info()

In [ ]:
oil.info()

In [ ]:
stores.info()

In [ ]:
train.info()

In [ ]:
test.info()

## Tidy Data

In [ ]:
# impute missing values in oil dataframe
oil['dcoilwtico']= oil['dcoilwtico'].interpolate(limit_direction='both')
oil.info()

## Join Dataframes

In [ ]:
# join data to make train dataset
train = train.copy(deep=True)
train = train.merge(oil, how='left', on='date', copy=False)
train = train.merge(stores[['store_nbr', 'city']], how='left', on='store_nbr')
train = train.merge(holiday_events[['date', 'type', 'locale_name']], how='left', left_on=['date', 'city'], right_on=['date', 'locale_name'], copy=False)
train = train[~train.date.isin(['2013-01-01'])]
train['is_holiday'] = train['type'].notnull()
train

In [ ]:
# join data to make test dataset
test = test.copy(deep=True)
test = test.merge(oil, how='left', on='date', copy=False)
test = test.merge(stores[['store_nbr', 'city']], how='left', on='store_nbr')
test = test.merge(holiday_events[['date', 'type', 'locale_name']], how='left', left_on=['date', 'city'], right_on=['date', 'locale_name'], copy=False)
test['is_holiday'] = test['type'].notnull()
test

In [ ]:
# feature lists for lightGBM model
target = 'sales'
predictor = ['store_nbr', 'family', 'onpromotion', 'year', 'month', 'day', 'dayofweek', 'dcoilwtico', 'is_holiday', 'ARIMA'] 
bool_predictor = ['is_holiday']
num_predictor = ['onpromotion', 'dcoilwtico', 'ARIMA'] 
cat_predictor = ['store_nbr', 'family', 'year', 'month', 'day', 'dayofweek']

## EDA

In [ ]:
# plot sales by family
fig_line = px.line(train, x='date', y='sales', color='family')

In [ ]:
fig_line.show()

In [ ]:
def test_stationarity(timeseries: pd.Series):
    '''
    INPUT: pandas timeseries of target
    
    Plots raw data, rolling mean, and rolling standard deviation
    
    '''
    rolmean = pd.Series(timeseries).rolling(window=12).mean() * 2
    rolstd = pd.Series(timeseries).rolling(window=12).std()
    
    fig, ax = plt.subplots(figsize=(16,4))
    ax.plot(timeseries, label='raw data')
    ax.plot(rolmean, label='rolling mean (x2)')
    ax.plot(rolstd, label='rolling std')
    ax.legend()

In [ ]:
test_stationarity(train['sales'])

In [ ]:
def ADF_test(timeseries: pd.DataFrame, dataDesc: str):
    '''
    Input: timeseires - pandas series of target
           dataDesc - string description of target
    
    Returns: Confidence that data is stationary at confidence level
    '''
    timeseries = timeseries.sample(frac=0.25)
    print(' > Is the {} stationary ?'.format(dataDesc))
    dftest = adfuller(timeseries.dropna(), autolag='AIC')
    print('Test statistic = {:.3f}'.format(dftest[0]))
    print('P-value = {:.3f}'.format(dftest[1]))
    print('Critical values :')
    for k, v in dftest[4].items():
        print('\t{}: {} - The data is {} stationary with {}% confidence'.format(k, v, 'not' if v<dftest[0] else '', 100-int(k[:-1])))

In [ ]:
ADF_test(train['sales'], 'raw data')

## Model

### ARIMA

Create an ARIMA prediction to use as a feature in the lightGBM model.

In [ ]:
arima_data = train.groupby(['date'])['sales'].sum().asfreq('d')
arima_train, arima_test = train_test_split(arima_data, test_size=0.33, random_state=42, shuffle=False)

Plot seasonal decomposition for using additive and multiplicative assumptions. The additive model shows the best decomposition so will assume this behaviour for ARIMA model.

In [ ]:
print(f'Additive model decomposition')
decomp_add = seasonal_decompose(arima_train.asfreq('MS'), model='additive')
decomp_add.plot()

In [ ]:
print(f'Multiplicative model decomposition')
decomp_mul = seasonal_decompose(arima_train.asfreq('MS'), model='multiplicative')
decomp_mul.plot()

In [ ]:
# Create possible combination of order parameteres
p = d = q = range(0,4)
pdq = list(itertools.product(p, d, q))
print('Examples of parameter combinations for Seasonal ARIMA: ')
print('SARIMAX: {}'.format(pdq[1]))
print('SARIMAX: {}'.format(pdq[1]))
print('SARIMAX: {}'.format(pdq[2]))
print('SARIMAX: {}'.format(pdq[2]))

In [ ]:
# For each possible order param, calculate AIC score
for param in pdq:
    mod = sm.tsa.statespace.SARIMAX(arima_train,
                                    order=param,
                                    enforce_stationarity=False,
                                    enforce_invertibility=False
                                   )
    results = mod.fit()
    print('ARIMA{} - AIC:{}'.format(param, results.aic))

The order parameters with the best AIC score were (3, 1, 3)

In [ ]:
model_sarima = sm.tsa.statespace.SARIMAX(arima_train,
                                order=(3, 1, 3),
                                enforce_stationarity=False,
                                enforce_invertibility=False
                                        )
results_sarima = model_sarima.fit()
print(results_sarima.summary().tables[1])

In [ ]:
results_sarima.plot_diagnostics(figsize=(10, 10))
plt.show()

In [ ]:
# create predictions covering timespan of data to use as feature
arima_pred = results_sarima.predict(0,1703)

### LightGBM

In [ ]:
# merge ARIMA feature to data
arima_pred = pd.DataFrame(arima_pred).reset_index()
arima_pred.columns = ['date', 'ARIMA']
train = train.merge(pd.DataFrame(arima_pred), how='left', on='date', copy=False)
test = test.merge(pd.DataFrame(arima_pred), how='left', on='date', copy=False)

In [ ]:
lgb_train, lgb_test = train_test_split(train, test_size=0.33, random_state=42, shuffle=False)

In [ ]:
def prep_light_gbm(df: pd.DataFrame, features: list, categorical_features: list, num_features: list, boolean_features: list):
    '''
    Takes a pandas dataframe and ensures that data types are correct for lightGBM model. Will return string
    if unexpected datatype.
    '''
    for col in features:
        if col in categorical_features:
            df[col] = df[col].astype(str).astype("category")
        elif col in boolean_features:
            df[col] = df[col].astype("category")
        elif col in num_features:
            df[col] = df[col].astype('float64')
        else:
            print(f"Other type detected for: {col}")
    return df

In [ ]:
lgb_train = prep_light_gbm(lgb_train, predictor, cat_predictor, num_predictor, bool_predictor)
lgb_test = prep_light_gbm(lgb_test, predictor, cat_predictor, num_predictor, bool_predictor)
test = prep_light_gbm(test, predictor, cat_predictor, num_predictor, bool_predictor)

param = {
    'objective': 'regression',
    'boosting_type': 'goss',
    'metric': 'rmse'
}

In [ ]:
def light_gbm(train: pd.DataFrame, params: dict): # test: pd.DataFrame
    
    train, valid = train_test_split(train, test_size=0.15, random_state=42, shuffle=False)
    
    train_data = lgb.Dataset(
        train[predictor],
        label=train[target],
        feature_name = predictor,
        categorical_feature = cat_predictor,
        free_raw_data=True
    )
    
    valid_data = lgb.Dataset(
        valid[predictor],
        label=valid[target],
        feature_name = predictor,
        categorical_feature = cat_predictor,
        free_raw_data=True
    )
    
    lgb_model = lgb.train(
        param,
        train_data,
        valid_sets=[valid_data],
        early_stopping_rounds=100
    )
    
    return lgb_model


In [ ]:
lgb_model = light_gbm(lgb_train, param)

In [ ]:
lgb_pred = lgb_model.predict(lgb_test[predictor])

In [ ]:
print('The rmse of prediction is:', round(mean_squared_error(lgb_test[target], lgb_pred) ** 0.5, 5))

In [ ]:
print('The R2 of prediction is:', round(r2_score(lgb_test[target], lgb_pred), 5))

In [ ]:
lgb.plot_importance(lgb_model)

## Kaggle Submission

In [ ]:
kaggle_prediction = lgb_model.predict(test[predictor])

In [ ]:
kaggle_submission = pd.DataFrame(kaggle_prediction)
kaggle_submission.rename(columns={0:'sales'},inplace=True)
kaggle_submission.index=test['id']

In [ ]:
# kaggle_submission.to_csv('submission.csv')

In [ ]:
kaggle_submission